# Analyse de l'Impact des Taux d'Intérêt sur les Choix de Consommation

## Introduction
Ce notebook a pour objectif d'explorer la relation entre les variations des taux d'intérêt et les décisions de consommation des ménages. Nous allons analyser différentes catégories de dépenses de consommation et les confronter à l'évolution des taux d'intérêt directeurs.

## 1. Importation des Librairies Nécessaires

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Configuration de Plotly pour le mode offline si nécessaire
# import plotly.offline as pyo
# pyo.init_notebook_mode(connected=True)

## 2. Chargement des Données
Nous allons commencer par charger les données sur les taux d'intérêt et les données sur les ventes au détail.

### 2.1 Taux d'Intérêt (Federal Funds Effective Rate)

In [2]:
DATA_PATH = 'data/'
interest_rate_file = os.path.join(DATA_PATH, 'Federal_Funds_Effective_Rate.csv')
df_interest_rate = pd.read_csv(interest_rate_file)
df_interest_rate.head()

,observation_date,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83


### 2.2 Données de Consommation (Ventes au Détail)
Nous allons charger plusieurs fichiers de ventes au détail pour différentes catégories.

In [3]:
consumption_files = {
    'clothing': 'Advance Retail Sales: Clothing and Clothing Accessory Stores.csv',
    'gas_stations': 'Advance Retail Sales: Gasoline Stations.csv',
    'electronics': 'Retail Sales: Electronics and Appliance Stores.csv',
    'jewelry': 'Retail Sales: Jewelry Stores.csv',
    'restaurants': 'Retail Sales: Restaurants and Other Eating Places.csv',
    'supermarkets': 'Retail Sales: Supermarkets and Other Grocery (Except Convenience) Stores.csv',
    'hobby_toy_game': 'Retail Sales: Hobby, Toy, and Game Stores.csv'
}

dfs_consumption = {}
for key, filename in consumption_files.items():
    file_path = os.path.join(DATA_PATH, filename)
    dfs_consumption[key] = pd.read_csv(file_path)
    print(f"Chargement de {key}: {filename}")
    print(dfs_consumption[key].head())
    print("---_---")

Chargement de clothing: Advance Retail Sales: Clothing and Clothing Accessory Stores.csv
  observation_date  RSCCAS
0       1992-01-01    9716
1       1992-02-01    9712
2       1992-03-01    9669
3       1992-04-01    9862
4       1992-05-01    9755
---_---
Chargement de gas_stations: Advance Retail Sales: Gasoline Stations.csv
  observation_date  RSGASS
0       1992-01-01   12803
1       1992-02-01   12601
2       1992-03-01   12639
3       1992-04-01   12710
4       1992-05-01   12870
---_---
Chargement de electronics: Retail Sales: Electronics and Appliance Stores.csv
  observation_date  MRTSSM443USS
0       1992-01-01          3874
1       1992-02-01          3904
2       1992-03-01          3916
3       1992-04-01          3958
4       1992-05-01          3981
---_---
Chargement de jewelry: Retail Sales: Jewelry Stores.csv
  observation_date  MRTSSM44831USS
0       1992-01-01            1248
1       1992-02-01            1230
2       1992-03-01            1283
3       1992-04-01 

## 3. Prétraitement et Nettoyage des Données
Cette section sera dédiée à la préparation des données pour l'analyse :
- Conversion des dates au format datetime
- Renommage des colonnes pour plus de clarté
- Définition de la date comme index
- Fusion des datasets sur la base des dates

In [4]:
# Prétraitement de df_interest_rate
print("Prétraitement du DataFrame des taux d'intérêt...")
df_interest_rate = pd.read_csv(interest_rate_file) # Re-read to ensure clean state for this cell
df_interest_rate['DATE'] = pd.to_datetime(df_interest_rate['observation_date'])
df_interest_rate = df_interest_rate.rename(columns={'FEDFUNDS': 'interest_rate'})
df_interest_rate = df_interest_rate.set_index('DATE')
if 'observation_date' in df_interest_rate.columns:
    df_interest_rate = df_interest_rate.drop(columns=['observation_date'])
print("DataFrame des taux d'intérêt prétraité :")
print(df_interest_rate.head())
print(df_interest_rate.info())
print("-------------------------------------\n")

# Prétraitement des DataFrames de consommation
processed_dfs_consumption = {}
for key, df_original in dfs_consumption.items():
    print(f"Prétraitement du DataFrame de consommation: {key}...")
    df = df_original.copy() # Travailler sur une copie
    df['DATE'] = pd.to_datetime(df['observation_date'])
    value_col_name = df.columns[1] # Nom original de la colonne de valeur
    df = df.rename(columns={value_col_name: f'sales_{key}'})
    df = df.set_index('DATE')
    # S'assurer de ne garder que la colonne de ventes pour éviter les colonnes dupliquées comme 'observation_date' si elles existent
    processed_dfs_consumption[key] = df[[f'sales_{key}']] 
    print(f"DataFrame {key} prétraité :")
    print(processed_dfs_consumption[key].head())
    print(processed_dfs_consumption[key].info())
    print("-------------------------------------\n")

# Fusion des données
print("Fusion des DataFrames...")
df_merged = df_interest_rate.copy()
for key, df_consump in processed_dfs_consumption.items():
    df_merged = df_merged.join(df_consump, how='outer')

if 'observation_date' in df_merged.columns: # Au cas où elle serait réintroduite par une colonne de consommation nommée ainsi
    df_merged = df_merged.drop(columns=['observation_date'])
    print("Colonne 'observation_date' supprimée de df_merged après jointures.")

print("\nAperçu du DataFrame fusionné (avant sélection de période et ffill) :")
print(df_merged.head())
print(f"\nNombre de NaNs par colonne après fusion (avant sélection et ffill):\n{df_merged.isnull().sum()}")

# Définition de la période d'analyse et slicing
start_date = '2001-01-01' # MODIFIÉ: pour correspondre au début de sales_supermarkets
end_date = '2024-11-01' 
print(f"\nRestriction du DataFrame à la période : {start_date} à {end_date}\n")

print("Application de ffill sur df_analysis...")
df_analysis = df_merged.loc[start_date:end_date].copy()

df_analysis = df_analysis.ffill()

print(f"\nShape du DataFrame d'analyse ({start_date} - {end_date}): {df_analysis.shape}")
print(f"\nNombre de NaNs dans le DataFrame d'analyse (df_analysis) après ffill:\n{df_analysis.isnull().sum()}")
print("\nDébut du DataFrame d'analyse (df_analysis):")
print(df_analysis.head())
print("\nFin du DataFrame d'analyse (df_analysis):")
print(df_analysis.tail())
print("\nInformations sur le DataFrame d'analyse (df_analysis) :")
print(df_analysis.info())

Prétraitement du DataFrame des taux d'intérêt...
DataFrame des taux d'intérêt prétraité :
            interest_rate
DATE                     
1954-07-01           0.80
1954-08-01           1.22
1954-09-01           1.07
1954-10-01           0.85
1954-11-01           0.83
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847 entries, 1954-07-01 to 2025-01-01
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   interest_rate  847 non-null    float64
dtypes: float64(1)
memory usage: 13.2 KB
None
-------------------------------------

Prétraitement du DataFrame de consommation: clothing...
DataFrame clothing prétraité :
            sales_clothing
DATE                      
1992-01-01            9716
1992-02-01            9712
1992-03-01            9669
1992-04-01            9862
1992-05-01            9755
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 396 entries, 1992-01-01 to 2024-12-01
Data columns (total 1

### Récapitulatif des Traitements de Données

1.  **Chargement des Données :**
    *   Les données sur le taux effectif des fonds fédéraux (`Federal_Funds_Effective_Rate.csv`) ont été chargées.
    *   Plusieurs jeux de données concernant les ventes au détail pour différentes catégories (vêtements, stations-service, électronique, bijouteries, restaurants, supermarchés, magasins de jouets/loisirs) ont été chargés.

2.  **Prétraitement Individuel des DataFrames :**
    *   **Conversion des Dates :** Pour chaque DataFrame, la colonne `DATE` (ou `observation_date` initialement) a été convertie au format `datetime` de pandas. Ceci est essentiel pour toute analyse de série temporelle.
    *   **Renommage des Colonnes :**
        *   Dans le DataFrame des taux d'intérêt, la colonne `FEDFUNDS` a été renommée `interest_rate` pour plus de clarté.
        *   Pour chaque DataFrame de consommation, la colonne contenant les valeurs des ventes a été renommée de manière standardisée (ex: `sales_clothing`).
    *   **Définition de l'Index :** La colonne `DATE` (convertie) a été définie comme index pour tous les DataFrames.
    *   **Nettoyage de Colonnes Redondantes :** La colonne `observation_date` (si présente après l'indexation) a été supprimée des DataFrames individuels pour éviter la redondance.

3.  **Fusion des DataFrames :**
    *   Tous les DataFrames prétraités ont été fusionnés en un unique DataFrame `df_merged`.
    *   Une jointure externe (`how='outer'`) a été utilisée pour conserver toutes les observations, introduisant des `NaN` là où les périodes ne se chevauchaient pas.

4.  **Définition de la Période d'Analyse Commune :**
    *   Une période d'analyse commune a été définie : du **1er janvier 2001 au 1er novembre 2024**.
    *   Ce choix a été fait car les données pour `sales_supermarkets` commencent en janvier 2001, assurant ainsi des données réelles pour toutes les séries au début de la période d'analyse.
    *   Le DataFrame `df_merged` a été filtré pour cette période, créant `df_analysis`.

5.  **Gestion des Valeurs Manquantes dans `df_analysis` :**
    *   La méthode `ffill()` (forward fill) a été appliquée à `df_analysis`. Elle propage la dernière observation valide pour remplir les `NaN` suivants.
    *   **Justification :** Pour des données mensuelles, `ffill()` est une approche raisonnable.
    *   **Cas spécifique de `sales_jewelry` :** Les données s'arrêtent en février 2021. Avec `ffill()`, les valeurs de mars 2021 à novembre 2024 sont une propagation de la valeur de février 2021. Ceci est une imputation à noter.
    *   Après ces étapes, `df_analysis` ne contient plus de valeurs manquantes pour la période 2001-01-01 à 2024-11-01.

Le DataFrame `df_analysis` est maintenant prêt pour l'analyse exploratoire.

## 4. Analyse Exploratoire des Données (EDA)
Ici, nous visualiserons les données pour comprendre les tendances, les saisonnalités et les corrélations potentielles.

In [5]:
# Assurez-vous que df_analysis est bien défini et nettoyé après l'exécution de la section 3.
# Si ce n'est pas le cas, ré-exécutez la cellule de prétraitement.
print("Vérification de df_analysis pour l'EDA:\n")
if 'df_analysis' in locals():
    print(df_analysis.head())
    print(df_analysis.info())
    print(f"Période de df_analysis: {df_analysis.index.min()} à {df_analysis.index.max()}")
    print(f"Nombre de NaNs dans df_analysis:\n{df_analysis.isnull().sum().sum()}")
else:
    print("df_analysis n'est pas défini. Veuillez exécuter la section de prétraitement des données.")

# 1. Visualisation du Taux d'Intérêt
print("\n--- Visualisation du Taux d'Intérêt ---")
fig_interest_rate = px.line(df_analysis, x=df_analysis.index, y='interest_rate', title='Évolution du Taux d\'Intérêt Effectif des Fonds Fédéraux')
fig_interest_rate.update_layout(xaxis_title='Date', yaxis_title='Taux d\'Intérêt (%)')
fig_interest_rate.show()

# 2. Visualisation des Ventes au Détail (chaque catégorie)
print("\n--- Visualisation des Ventes au Détail (par catégorie) ---")
sales_columns = [col for col in df_analysis.columns if 'sales_' in col]

for sales_col in sales_columns:
    fig_sales = px.line(df_analysis, x=df_analysis.index, y=sales_col, title=f'Évolution des Ventes: {sales_col.replace("sales_", "").replace("_", " ").title()}')
    fig_sales.update_layout(xaxis_title='Date', yaxis_title='Ventes (Millions de Dollars)')
    fig_sales.show()

# 3. Superposition Taux d'Intérêt et Ventes
print("\n--- Superposition Taux d'Intérêt et Ventes (par catégorie) ---")
for sales_col in sales_columns:
    fig_overlay = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Ajouter la série des ventes
    fig_overlay.add_trace(
        go.Scatter(x=df_analysis.index, y=df_analysis[sales_col], name=f'Ventes: {sales_col.replace("sales_", "").title()}'),
        secondary_y=False,
    )
    
    # Ajouter la série des taux d'intérêt
    fig_overlay.add_trace(
        go.Scatter(x=df_analysis.index, y=df_analysis['interest_rate'], name='Taux d\'Intérêt', line=dict(dash='dot')),
        secondary_y=True,
    )
    
    # Titres et légendes
    fig_overlay.update_layout(
        title_text=f'Ventes ({sales_col.replace("sales_", "").title()}) vs. Taux d\'Intérêt'
    )
    fig_overlay.update_xaxes(title_text='Date')
    fig_overlay.update_yaxes(title_text=f'<b>Ventes: {sales_col.replace("sales_", "").title()}</b> (Millions de Dollars)', secondary_y=False)
    fig_overlay.update_yaxes(title_text='<b>Taux d\'Intérêt</b> (%)', secondary_y=True)
    
    fig_overlay.show()

# 4. Matrice de Corrélation
print("\n--- Matrice de Corrélation ---")
correlation_matrix = df_analysis.corr()

fig_corr = px.imshow(correlation_matrix, 
                     text_auto=True, 
                     aspect="auto",
                     color_continuous_scale='RdBu_r', # Rouge-Bleu inversé
                     title='Matrice de Corrélation des Variables d\'Analyse')
fig_corr.show()

print("\nMatrice de Corrélation (valeurs numériques) :")
print(correlation_matrix)


Vérification de df_analysis pour l'EDA:

            interest_rate  sales_clothing  sales_gas_stations  \
DATE                                                            
2001-01-01           5.98         14135.0             21931.0   
2001-02-01           5.49         14166.0             21615.0   
2001-03-01           5.31         13801.0             20652.0   
2001-04-01           4.80         14217.0             21618.0   
2001-05-01           4.21         13950.0             22628.0   

            sales_electronics  sales_jewelry  sales_restaurants  \
DATE                                                              
2001-01-01             7376.0         2062.0            20613.0   
2001-02-01             7334.0         2057.0            20547.0   
2001-03-01             7267.0         2062.0            23239.0   
2001-04-01             7212.0         2016.0            22217.0   
2001-05-01             7135.0         1957.0            23460.0   

            sales_supermarkets  s


--- Visualisation des Ventes au Détail (par catégorie) ---



--- Superposition Taux d'Intérêt et Ventes (par catégorie) ---



--- Matrice de Corrélation ---



Matrice de Corrélation (valeurs numériques) :
                      interest_rate  sales_clothing  sales_gas_stations  \
interest_rate              1.000000        0.089700            0.033951   
sales_clothing             0.089700        1.000000            0.855741   
sales_gas_stations         0.033951        0.855741            1.000000   
sales_electronics          0.117262        0.061198            0.113638   
sales_jewelry              0.217093        0.918301            0.800901   
sales_restaurants          0.186656        0.893617            0.793298   
sales_supermarkets         0.068110        0.814428            0.784689   
sales_hobby_toy_game       0.022664        0.246908            0.170157   

                      sales_electronics  sales_jewelry  sales_restaurants  \
interest_rate                  0.117262       0.217093           0.186656   
sales_clothing                 0.061198       0.918301           0.893617   
sales_gas_stations             0.113638       

### Observations Détaillées de l'Analyse Exploratoire des Données (EDA)

Cette section détaille les observations tirées de l'analyse de la matrice de corrélation et de l'examen attendu des graphiques de séries temporelles pour la période allant de janvier 2001 à novembre 2024.

#### 1. Analyse de la Matrice de Corrélation

La matrice de corrélation de Pearson a été calculée pour quantifier les relations linéaires entre le taux d'intérêt et les différentes catégories de ventes, ainsi qu'entre les catégories de ventes elles-mêmes.

##### 1.1. Taux d'Intérêt (`interest_rate`) vs. Catégories de Ventes

De manière générale, les corrélations linéaires directes entre les variations mensuelles du taux d'intérêt effectif des fonds fédéraux et les ventes au détail sont faibles. Cela suggère que la relation, si elle existe de manière significative, n'est probablement pas simple, directe et linéaire, ou qu'elle est masquée par d'autres facteurs économiques plus dominants ou qu'elle opère avec des décalages.

*   **`sales_jewelry` (Ventes de Bijoux) : Corrélation de 0.217**
    *   C'est la corrélation positive la plus "élevée" avec les taux d'intérêt parmi les catégories de ventes. Ce résultat est quelque peu contre-intuitif, car on pourrait s'attendre à ce que des taux plus élevés (augmentant le coût du crédit et signalant potentiellement un ralentissement économique) freinent les dépenses pour des biens de luxe.
    *   *Explications Possibles :*
        1.  **Facteurs Confondants :** Des périodes de croissance économique robuste pourraient entraîner simultanément une augmentation des dépenses de luxe et une hausse des taux d'intérêt par la banque centrale pour prévenir la surchauffe.
        2.  **Inflation :** En période d'inflation, les prix des bijoux augmentent (gonflant les chiffres de ventes) et les taux d'intérêt sont souvent relevés pour contrer cette inflation.
        3.  **Richesse et Inégalités :** La clientèle pour les bijoux de luxe pourrait être moins sensible aux variations des taux d'intérêt que la population générale.
        4.  **Effet de `ffill` :** Il est crucial de noter que les données de `sales_jewelry` sont propagées par `ffill` à partir de février 2021. Cela pourrait affecter la corrélation calculée sur l'ensemble de la période si les dynamiques des taux d'intérêt et des ventes réelles de bijoux ont divergé après cette date.

*   **`sales_restaurants` (Ventes des Restaurants) : Corrélation de 0.187**
    *   Une corrélation positive modérée, similaire à celle des bijoux. Les sorties au restaurant sont des dépenses discrétionnaires.
    *   *Explications Possibles :* Similaires à celles pour les bijoux (croissance économique, confiance des consommateurs entraînant plus de dépenses et potentiellement des taux plus élevés).

*   **`sales_electronics` (Ventes d'Électronique) : Corrélation de 0.117**
    *   Corrélation positive faible. Les achats d'électronique peuvent être importants et parfois financés, mais la corrélation ne montre pas une forte sensibilité négative aux taux. L'innovation, les cycles de produits et les promotions jouent probablement un rôle plus important.

*   **`sales_clothing` (Ventes de Vêtements) : Corrélation de 0.090**
    *   Corrélation positive faible. Les vêtements sont une dépense plus courante, mais avec une composante discrétionnaire.

*   **`sales_supermarkets` (Ventes des Supermarchés) : Corrélation de 0.068**
    *   Corrélation positive très faible. En tant que dépenses essentielles, elles devraient être moins sensibles aux taux d'intérêt.

*   **`sales_gas_stations` (Ventes des Stations-Service) : Corrélation de 0.034**
    *   Corrélation quasi nulle. Les ventes sont principalement dictées par les prix du pétrole et les besoins de déplacement.

*   **`sales_hobby_toy_game` (Ventes de Jouets, Jeux, Loisirs) : Corrélation de 0.023**
    *   Corrélation quasi nulle. Dépenses discrétionnaires, mais pas typiquement de gros achats financés par crédit.

**Conclusion sur les Taux d'Intérêt et les Ventes :** L'absence de corrélations négatives fortes suggère que l'impact direct et immédiat d'une hausse des taux sur une baisse des ventes (ou vice-versa) n'est pas un phénomène dominant au niveau agrégé mensuel. Les graphiques superposés seront essentiels pour déceler des dynamiques plus complexes.

##### 1.2. Corrélations entre les Catégories de Ventes

*   **Fortes Corrélations Positives (souvent > 0.80) :**
    *   `sales_restaurants` et `sales_supermarkets` (0.960) : Très forte. Logique, car les deux sont liés à l'alimentation et aux dépenses courantes des ménages.
    *   `sales_clothing` et `sales_jewelry` (0.918)
    *   `sales_clothing` et `sales_restaurants` (0.894)
    *   `sales_clothing` et `sales_gas_stations` (0.856)
    *   `sales_clothing` et `sales_supermarkets` (0.814)
    *   `sales_jewelry` et `sales_restaurants` (0.857)
    *   Ces fortes corrélations indiquent que les dépenses dans ces secteurs tendent à évoluer de concert. Cela peut être dû à des facteurs macroéconomiques communs : croissance économique générale, inflation (qui gonfle les chiffres de ventes nominales), confiance des consommateurs, et augmentation générale du niveau de vie sur la période.

*   **Corrélations Négatives Notables :**
    *   `sales_electronics` et `sales_supermarkets` (-0.397)
    *   `sales_electronics` et `sales_restaurants` (-0.280)
    *   Ces corrélations négatives, bien que modérées, sont intéressantes. Elles pourraient suggérer un arbitrage budgétaire des consommateurs (par exemple, après un gros achat électronique, le budget pour les restaurants ou certains achats de supermarché est temporairement réduit) ou des préférences de consommation qui évoluent différemment. Par exemple, une période de forte innovation et d'achats en électronique pourrait coïncider avec une tendance à moins dépenser à l'extérieur.

#### 2. Analyse Visuelle Attendue des Graphiques de Séries Temporelles

##### 2.1. Taux d'Intérêt (`taux d'interets.png`)

*   Le graphique devrait clairement illustrer les cycles de la politique monétaire :
    *   Baisse post-bulle internet et 11 septembre.
    *   Remontée jusqu'à la crise financière de 2008.
    *   Chute drastique à la "Zero Lower Bound" (ZLB) après 2008, suivie d'une longue période de taux quasi nuls.
    *   Tentative de normalisation pré-pandémie.
    *   Retour à la ZLB en réponse à la pandémie de COVID-19 en 2020.
    *   Hausse très rapide et significative à partir de 2022 pour lutter contre l'inflation.
    *   Ces régimes de taux distincts sont cruciaux pour interpréter les réactions des ventes.

##### 2.2. Ventes au Détail - Tendances Générales et Saisonnalité (Graphiques Individuels)

*   **Tendance Générale Commune :** La plupart des catégories de ventes devraient afficher une croissance nominale sur la période 2001-2024, due à l'inflation, à la croissance démographique et à la croissance économique. Des impacts négatifs visibles sont attendus lors de la récession de 2008 et surtout lors des confinements de 2020 pour certaines catégories.
*   **`ventes colthing.png`**: Croissance avec forte saisonnalité (pics en novembre-décembre, potentiellement au printemps/été pour les nouvelles collections).
*   **`vente gas station.png`**: Forte volatilité due aux prix du pétrole, superposée à une croissance de la demande. Saisonnalité possible (été).
*   **`vente electronics.png`**: Croissance, peut-être avec des cycles de produits. Saisonnalité en fin d'année.
*   **`ventes jewlery.png` / `jewlary.png`**: Croissance jusqu'en février 2021, puis ligne plate (effet `ffill`). Forte saisonnalité en décembre.
*   **`vente restaurents.png`**: Croissance, avec une chute spectaculaire en 2020 et une reprise ensuite. Saisonnalité possible (vacances).
*   **`ventes supermarkets.png`**: Croissance la plus stable et régulière. Moins de volatilité. Saisonnalité mineure (pics avant les fêtes).
*   **`vente toy games hobby.png`**: Croissance. Très forte saisonnalité en novembre-décembre. Possible pic d'activité pendant les confinements de 2020.

##### 2.3. Analyse des Graphiques Superposés (Ventes vs. Taux d'Intérêt)

Ces graphiques sont essentiels pour aller au-delà des corrélations linéaires simples.

*   **Objectif Général :** Identifier visuellement si des changements de régime des taux d'intérêt (hausses, baisses, périodes de stabilité) coïncident avec des changements de tendance ou de volatilité dans les ventes, même avec un décalage.
*   **Points d'Attention Spécifiques :**
    *   **Périodes de Baisse des Taux (ex: 2001-2004, 2008-2009, 2020) :** Observe-t-on une accélération des ventes (avec ou sans décalage) ? Laquelle des catégories semble la plus réactive ?
    *   **Périodes de Hausse des Taux (ex: 2004-2006, 2016-2018, 2022-présent) :** Observe-t-on un ralentissement, une stagnation ou une baisse des ventes ?
    *   **Périodes de Taux Stables (ex: ZLB 2009-2015) :** Comment les ventes évoluent-elles lorsque la politique de taux est "inactive" ? D'autres facteurs deviennent-ils plus visibles ?
    *   **Relation avec `sales_jewelry` et `sales_restaurants` (corrélations positives) :** Le graphique permettra de voir si cette corrélation est constante ou si elle est tirée par des périodes spécifiques où, par exemple, une forte croissance économique a conduit à la fois à une augmentation des dépenses discrétionnaires et à une politique monétaire plus restrictive.
    *   **Décalages (Lags) :** L'effet des taux sur l'économie réelle et la consommation n'est généralement pas instantané. Il faut chercher si un changement de taux est suivi d'un changement dans les ventes quelques mois plus tard.

#### 3. Synthèse et Implications

*   **Complexité de la Relation :** L'EDA suggère que la relation entre les taux d'intérêt directeurs et les dépenses de consommation n'est pas simple et directe. Les faibles corrélations linéaires masquent probablement des dynamiques plus complexes, non linéaires, ou dépendantes du régime économique.
*   **Importance des Facteurs Confondants :** L'inflation, la croissance du PIB, le chômage, la confiance des consommateurs, les chocs exogènes (pandémie, crises géopolitiques affectant les prix de l'énergie) jouent tous un rôle majeur et peuvent dominer ou interagir avec l'impact des taux d'intérêt.
*   **Décalages Temporels :** La politique monétaire agit avec des délais. L'analyse visuelle des graphiques superposés est une première étape pour identifier des décalages potentiels, mais des techniques de séries temporelles plus avancées (comme l'analyse de causalité de Granger ou les modèles vectoriels autorégressifs avec retards - VAR/VECM) seraient nécessaires pour les quantifier.
*   **Hétérogénéité par Secteur :** Les différentes catégories de ventes peuvent réagir différemment aux variations des taux, en fonction de leur nature (essentielle vs. discrétionnaire), de leur coût unitaire, et de la sensibilité de leur clientèle au crédit.
*   **Caveat pour `sales_jewelry` :** L'imputation `ffill` pour les données de ventes de bijoux à partir de février 2021 signifie que toute analyse de cette variable pour la période récente doit être interprétée avec une extrême prudence, en particulier si on la compare aux taux d'intérêt qui ont fortement évolué durant cette même période.

En conclusion, si l'objectif est de modéliser l'impact des taux d'intérêt sur la consommation, une approche simpliste basée sur des corrélations directes sera insuffisante. Il faudra envisager des modèles qui peuvent capturer des dynamiques plus riches, inclure d'autres variables économiques pertinentes, et tenir compte des décalages temporels.

## 5. Analyse des Décalages Temporels (Lag Analysis)

L'analyse exploratoire des données (EDA) a montré des corrélations instantanées (lag 0) généralement faibles entre le taux d'intérêt et les différentes catégories de ventes. Cependant, l'impact des variations des taux d'intérêt sur la consommation n'est souvent pas immédiat et peut se manifester avec un certain décalage.

Cette section vise à identifier ces décalages potentiels en calculant les corrélations croisées entre la série des taux d'intérêt (décalée dans le temps) et chaque série de ventes.

Nous allons :
1. Définir une plage de décalages (par exemple, de 0 à 24 mois) où le taux d'intérêt précède les ventes.
2. Pour chaque catégorie de ventes, calculer la corrélation entre les ventes au temps *t* et le taux d'intérêt au temps *t-lag*.
3. Visualiser ces corrélations en fonction du décalage pour chaque catégorie.
4. Identifier le décalage qui maximise la valeur absolue de la corrélation pour chaque catégorie de ventes.

**Note :** Un décalage positif ici signifie que le taux d'intérêt du passé est corrélé avec les ventes actuelles. Par exemple, un décalage de 6 signifie la corrélation entre les ventes(t) et le taux_intérêt(t-6).

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import pearsonr

# Assurez-vous que df_analysis est disponible et correctement chargé.
# Si ce n'est pas le cas, exécutez les cellules précédentes (notamment prétraitement et EDA).
if 'df_analysis' not in locals() or df_analysis.empty:
    print("df_analysis n'est pas défini ou est vide. Veuillez exécuter les cellules de prétraitement des données.")
    # Potentiellement, charger les données ici si nécessaire ou arrêter
else:
    print("df_analysis trouvé. Début de l'analyse des décalages.")
    # Définition de la plage de décalages à tester (en mois)
    # Un décalage de k signifie qu'on corrèle sales(t) avec interest_rate(t-k)
    max_lag = 24  # Tester jusqu'à 24 mois de décalage
    lags = range(0, max_lag + 1)

    sales_columns = [col for col in df_analysis.columns if 'sales_' in col]
    lag_analysis_results = {}

    print(f"Analyse des décalages de 0 à {max_lag} mois:\n")

    for sales_col in sales_columns:
        cross_correlations = []
        for lag in lags:
            shifted_interest_rate = df_analysis['interest_rate'].shift(lag)
            temp_df = pd.DataFrame({'sales': df_analysis[sales_col], 'shifted_interest_rate': shifted_interest_rate}).dropna()
            if len(temp_df) < 2:
                correlation = np.nan
            else:
                correlation, _ = pearsonr(temp_df['sales'], temp_df['shifted_interest_rate'])
            cross_correlations.append(correlation)
        
        lag_analysis_results[sales_col] = pd.Series(cross_correlations, index=lags)
        
        if lag_analysis_results[sales_col].notna().any():
            best_lag_idx = lag_analysis_results[sales_col].abs().idxmax()
            best_corr_value = lag_analysis_results[sales_col].loc[best_lag_idx]
            print(f"Résultats pour {sales_col.replace('sales_','').replace('_',' ').title()}: ")
            print(f"  Meilleure corrélation de {best_corr_value:.4f} trouvée à un décalage de {best_lag_idx} mois.")
        else:
            best_lag_idx = np.nan
            best_corr_value = np.nan
            print(f"Résultats pour {sales_col.replace('sales_','').replace('_',' ').title()}: ")
            print(f"  Impossible de calculer les corrélations (données insuffisantes ou toutes NaN).")
        
        # Visualisation des corrélations croisées pour cette catégorie
        fig_lag = px.bar(lag_analysis_results[sales_col], 
                           title=f'Corrélation Croisée: Ventes {sales_col.replace("sales_","").replace("_"," ").title()} vs. Taux d\'Intérêt Décalé',
                           labels={'index': 'Décalage du Taux d\'Intérêt (Mois)', 'value': 'Corrélation de Pearson'})
        fig_lag.update_layout(showlegend=False)
        if pd.notna(best_lag_idx):
             fig_lag.add_vline(x=best_lag_idx, line_width=2, line_dash="dash", line_color="red")
        fig_lag.show()

    # Affichage d'un résumé tabulaire des meilleurs décalages
    summary_lag_data = {
        'Catégorie de Ventes': [], 
        'Meilleur Décalage (Mois)': [], 
        'Corrélation au Meilleur Décalage': []
    }

    for sales_col, correlations in lag_analysis_results.items():
        if correlations.notna().any():
            best_lag_idx = correlations.abs().idxmax()
            best_corr_value = correlations.loc[best_lag_idx]
            summary_lag_data['Catégorie de Ventes'].append(sales_col.replace('sales_','').replace('_',' ').title())
            summary_lag_data['Meilleur Décalage (Mois)'].append(best_lag_idx)
            summary_lag_data['Corrélation au Meilleur Décalage'].append(f"{best_corr_value:.4f}")
        else:
            summary_lag_data['Catégorie de Ventes'].append(sales_col.replace('sales_','').replace('_',' ').title())
            summary_lag_data['Meilleur Décalage (Mois)'].append('N/A')
            summary_lag_data['Corrélation au Meilleur Décalage'].append('N/A')

    df_summary_lag = pd.DataFrame(summary_lag_data)
    print("\nRésumé des Meilleurs Décalages :")
    print(df_summary_lag.to_string())

df_analysis trouvé. Début de l'analyse des décalages.
Analyse des décalages de 0 à 24 mois:

Résultats pour Clothing: 
  Meilleure corrélation de -0.4522 trouvée à un décalage de 24 mois.


Résultats pour Gas Stations: 
  Meilleure corrélation de -0.4257 trouvée à un décalage de 24 mois.


Résultats pour Electronics: 
  Meilleure corrélation de 0.1204 trouvée à un décalage de 6 mois.


Résultats pour Jewelry: 
  Meilleure corrélation de -0.3101 trouvée à un décalage de 24 mois.


Résultats pour Restaurants: 
  Meilleure corrélation de -0.3680 trouvée à un décalage de 24 mois.


Résultats pour Supermarkets: 
  Meilleure corrélation de -0.3613 trouvée à un décalage de 24 mois.


Résultats pour Hobby Toy Game: 
  Meilleure corrélation de -0.1142 trouvée à un décalage de 24 mois.



Résumé des Meilleurs Décalages :
  Catégorie de Ventes  Meilleur Décalage (Mois) Corrélation au Meilleur Décalage
0            Clothing                        24                          -0.4522
1        Gas Stations                        24                          -0.4257
2         Electronics                         6                           0.1204
3             Jewelry                        24                          -0.3101
4         Restaurants                        24                          -0.3680
5        Supermarkets                        24                          -0.3613
6      Hobby Toy Game                        24                          -0.1142


# Tendances Principales Observées (Données Nominales)
*Ces observations sont basées sur l'analyse des corrélations croisées (données nominales) et ne tiennent pas compte de l'ajustement pour l'inflation.*

Pour une majorité significative des catégories de ventes analysées, l'impact le plus marqué des variations des taux d'intérêt se manifeste avec un décalage de 24 mois (2 ans). De plus, cette relation est négative, ce qui signifie qu'une augmentation des taux d'intérêt tend à être suivie d'une baisse des ventes dans ces secteurs deux ans plus tard (et inversement, une baisse des taux d'intérêt serait suivie d'une hausse des ventes).

## Catégories Suivant ce Modèle

### Clothing (Vêtements)
- **Meilleur Décalage** : 24 mois
- **Corrélation** : -0.4522
- **Analyse** : Les ventes de vêtements montrent la corrélation négative la plus forte avec les taux d'intérêt parmi toutes les catégories, avec un impact maximal observé après 2 ans. Cela suggère une sensibilité relativement élevée de ce secteur aux conditions de crédit ou au sentiment économique influencé par les taux.

### Gas Stations (Stations-Service)
- **Meilleur Décalage** : 24 mois
- **Corrélation** : -0.4257
- **Analyse** : Les ventes des stations-service présentent également une corrélation négative notable, de force comparable à celle des vêtements, avec le même décalage de 24 mois.

### Restaurants
- **Meilleur Décalage** : 24 mois
- **Corrélation** : -0.3680
- **Analyse** : Le secteur de la restauration réagit aussi négativement, avec une corrélation modérée, indiquant qu'une hausse des taux d'intérêt se répercute sur les dépenses en restaurant environ deux ans plus tard.

### Supermarkets (Supermarchés)
- **Meilleur Décalage** : 24 mois
- **Corrélation** : -0.3613
- **Analyse** : Similaire aux restaurants, les supermarchés voient leurs ventes affectées négativement par les hausses de taux d'intérêt avec un délai de 24 mois et une corrélation modérée.

### Jewelry (Bijouterie)
- **Meilleur Décalage** : 24 mois
- **Corrélation** : -0.3101
- **Analyse** : Les ventes de bijoux suivent la même tendance de réaction négative avec un décalage de 24 mois, bien que la force de la corrélation soit légèrement plus faible que les catégories précédentes mais reste modérée.

### Hobby Toy Game (Loisirs, Jouets, Jeux)
- **Meilleur Décalage** : 24 mois
- **Corrélation** : -0.1142
- **Analyse** : Bien que cette catégorie réagisse également avec un décalage de 24 mois et de manière négative, la corrélation est la plus faible de ce groupe. Cela suggère que les ventes de loisirs, jouets et jeux sont moins fortement (ou moins systématiquement) impactées négativement par les variations des taux d'intérêt que les autres secteurs mentionnés ci-dessus.

## Cas Particulier (Décalage Court, Corrélation Positive)

### Electronics (Électronique)
- **Meilleur Décalage** : 6 mois
- **Corrélation** : 0.1204
- **Analyse** : Le secteur de l'électronique se distingue nettement des autres. Le décalage optimal est beaucoup plus court (6 mois). De plus, la corrélation est positive, bien que faible. Cela indiquerait qu'une augmentation des taux d'intérêt est associée à une légère augmentation des ventes d'électronique six mois plus tard. Ce comportement est inverse à celui de toutes les autres catégories analysées et se manifeste plus rapidement.

## Conclusion Basée sur le Résumé Fourni

D'après votre résumé, la plupart des secteurs de vente au détail étudiés (vêtements, stations-service, restaurants, supermarchés, bijouterie, et dans une moindre mesure, les jeux/jouets/loisirs) semblent être impactés négativement par les hausses de taux d'intérêt, cet impact se matérialisant de façon la plus significative après un délai de deux ans. Le secteur de l'électronique fait figure d'exception, avec une réaction plus rapide (6 mois) et une corrélation positive faible, suggérant une dynamique différente face aux variations des taux d'intérêt. Les forces des corrélations varient, indiquant des sensibilités différentes d'un secteur à l'autre.


## 6. Tests de Causalité de Granger (Données Nominales différenciées)

L'analyse des décalages temporels a identifié les retards optimaux pour lesquels les taux d'intérêt présentent la plus forte corrélation avec chaque catégorie de ventes. Maintenant, nous allons utiliser les tests de causalité de Granger pour évaluer si les valeurs passées des taux d'intérêt ont un pouvoir prédictif statistiquement significatif sur les ventes actuelles, en tenant compte de ces décalages.

**Qu'est-ce que la Causalité de Granger ?**

La causalité de Granger est un concept statistique qui teste si une série temporelle X est utile pour prévoir une autre série temporelle Y. Si les valeurs passées de X (prises avec un certain nombre de décalages) réduisent significativement l'erreur de prédiction de Y par rapport à une prédiction basée uniquement sur les valeurs passées de Y, alors on dit que X "Granger-cause" Y.

Il est important de noter qu'il s'agit d'une notion de "causalité prédictive" et non d'une causalité au sens philosophique ou structurel. Un test de causalité de Granger positif n'implique pas nécessairement que X *cause* Y dans le monde réel, mais plutôt que X contient des informations utiles pour prédire Y.

**Hypothèses du test :**
*   **H0 (hypothèse nulle) :** Les coefficients des valeurs passées des taux d'intérêt sont nuls (c'est-à-dire que les taux d'intérêt ne Granger-causent pas les ventes).
*   **H1 (hypothèse alternative) :** Au moins un des coefficients des valeurs passées des taux d'intérêt est non nul (c'est-à-dire que les taux d'intérêt Granger-causent les ventes).

**Stationnarité :**
Les tests de causalité de Granger supposent que les séries temporelles utilisées sont stationnaires. Une série non stationnaire peut conduire à des résultats fallacieux (corrélations fallacieuses). Pour assurer la stationnarité, nous allons appliquer une différenciation d'ordre 1 aux séries de ventes et de taux d'intérêt avant d'effectuer les tests.

**Déroulement :**
1. Pour chaque catégorie de ventes :
   a. Préparer les données : la série de ventes et la série des taux d'intérêt.
   b. Appliquer une différenciation d'ordre 1 pour rendre les séries stationnaires.
   c. Effectuer le test de causalité de Granger pour déterminer si `interest_rate` Granger-cause la catégorie de ventes, en utilisant le `maxlag` identifié lors de l'analyse des décalages.
2. Analyser les p-values des tests F. Une p-value faible (typiquement < 0.05) suggère de rejeter l'hypothèse nulle, indiquant une causalité de Granger.

In [7]:
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd
import numpy as np

# Assurez-vous que df_analysis est disponible.
if 'df_analysis' not in locals() or df_analysis.empty:
    print("df_analysis n'est pas défini ou est vide. Veuillez exécuter les cellules précédentes.")
    # Potentiellement, charger les données ici si nécessaire ou arrêter
else:
    print("df_analysis trouvé. Début des tests de causalité de Granger.")

    # Mapping des noms de catégories de ventes (simplifiés) vers les noms de colonnes réels et leurs meilleurs décalages
    # Basé sur votre résumé : 
    # 0 Clothing: 24, 1 Gas Stations: 24, 2 Electronics: 6, 3 Jewelry: 24, 
    # 4 Restaurants: 24, 5 Supermarkets: 24, 6 Hobby Toy Game: 24
    sales_col_to_lag_map = {
        'sales_clothing': 24, 
        'sales_gas_stations': 24, 
        'sales_electronics': 6,  
        'sales_jewelry': 24, 
        'sales_restaurants': 24, 
        'sales_supermarkets': 24, 
        'sales_hobby_toy_game': 24 
    }

    granger_summary_results = []

    for sales_col, max_lag_val in sales_col_to_lag_map.items():
        if sales_col not in df_analysis.columns:
            print(f"Colonne {sales_col} non trouvée dans df_analysis. Passage au suivant.")
            granger_summary_results.append({
                'Catégorie': sales_col,
                'Max Lag Utilisé': max_lag_val,
                'Test Effectué': 'Non (Colonne Manquante)',
                'Min p-value (F-test)': np.nan,
                'Lags Significatifs (p<0.05)': 'N/A'
            })
            continue

        display_name = sales_col.replace('sales_','').replace('_',' ').title()
        print(f"\n--- Test de Causalité de Granger : Taux d'Intérêt -> {display_name} (max_lag={max_lag_val}) ---")
        
        # Préparation des données pour le test (ventes et taux d'intérêt)
        # Utilisation de .copy() pour éviter les SettingWithCopyWarning
        df_subset = df_analysis[[sales_col, 'interest_rate']].copy()
        
        # 1. Stationnarisation par différenciation d'ordre 1
        df_subset[sales_col] = df_subset[sales_col].diff()
        df_subset['interest_rate'] = df_subset['interest_rate'].diff()
        df_subset.dropna(inplace=True) # Supprimer les NaN créés par diff()

        min_p_value_for_category = 1.0
        significant_lags_for_category = []
        test_performed_flag = 'Non (Données Insuffisantes)'

        # Vérifier s'il y a suffisamment de données après différenciation
        # grangercausalitytests requiert len(df) > k * num_variables + 1 (approx) et aussi > max_lag
        if len(df_subset) > max_lag_val * 2 + 5: # Heuristique, k=nombre de variables (2 ici)
            try:
                # verbose=False pour parser nous-mêmes les résultats
                gc_results = grangercausalitytests(df_subset[[sales_col, 'interest_rate']], maxlag=max_lag_val, addconst=True, verbose=False)
                test_performed_flag = 'Oui'
                print(f"  P-values pour le F-test (Taux d'Intérêt Granger-causant {display_name}):")
                for lag_num in range(1, max_lag_val + 1):
                    # gc_results[lag_num] est une liste, le premier élément est un dict de résultats de test
                    # Le test F 'ssr_ftest' est (F, p, df_denom, df_num)
                    p_value = gc_results[lag_num][0]['ssr_ftest'][1]
                    print(f"    Lag {lag_num}: p-value = {p_value:.4f}")
                    if p_value < min_p_value_for_category:
                        min_p_value_for_category = p_value
                    if p_value < 0.05: # Seuil de significativité alpha = 0.05
                        significant_lags_for_category.append(lag_num)
                
                if not significant_lags_for_category:
                    print(f"  Aucune causalité de Granger significative (p < 0.05) détectée des taux d'Intérêt vers {display_name} jusqu'au lag {max_lag_val}.")
                else:
                    print(f"  Causalité de Granger significative (p < 0.05) détectée aux lags: {significant_lags_for_category}")
            except Exception as e:
                print(f"  Erreur lors du test de Granger pour {display_name} avec max_lag={max_lag_val}: {e}")
                min_p_value_for_category = np.nan # Marquer comme échec
                test_performed_flag = f'Non (Erreur: {e})'
        else:
            print(f"  Données insuffisantes pour {display_name} après différenciation (N={len(df_subset)}) pour max_lag={max_lag_val}. Test non effectué.")
            min_p_value_for_category = np.nan
        
        granger_summary_results.append({
            'Catégorie': display_name,
            'Max Lag Utilisé': max_lag_val,
            'Test Effectué': test_performed_flag,
            'Min p-value (F-test)': f"{min_p_value_for_category:.4f}" if pd.notna(min_p_value_for_category) else 'N/A',
            'Lags Significatifs (p<0.05)': str(significant_lags_for_category) if significant_lags_for_category else 'Aucun'
        })

    # Affichage du résumé des tests de Granger
    df_granger_summary = pd.DataFrame(granger_summary_results)
    print("\nRésumé des Tests de Causalité de Granger (Taux d'Intérêt -> Ventes) :")
    print(df_granger_summary.to_string())

df_analysis trouvé. Début des tests de causalité de Granger.

--- Test de Causalité de Granger : Taux d'Intérêt -> Clothing (max_lag=24) ---


/home/raouf/Bureau/TP S2 DAC/DALAS/TEMP_DALAS_NO_NAME/.venv/lib/python3.12/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results



  P-values pour le F-test (Taux d'Intérêt Granger-causant Clothing):
    Lag 1: p-value = 0.9747
    Lag 2: p-value = 0.0204
    Lag 3: p-value = 0.0593
    Lag 4: p-value = 0.0352
    Lag 5: p-value = 0.0501
    Lag 6: p-value = 0.0244
    Lag 7: p-value = 0.0128
    Lag 8: p-value = 0.0209
    Lag 9: p-value = 0.0179
    Lag 10: p-value = 0.0236
    Lag 11: p-value = 0.0399
    Lag 12: p-value = 0.0442
    Lag 13: p-value = 0.0630
    Lag 14: p-value = 0.0381
    Lag 15: p-value = 0.0659
    Lag 16: p-value = 0.1669
    Lag 17: p-value = 0.1514
    Lag 18: p-value = 0.1462
    Lag 19: p-value = 0.1694
    Lag 20: p-value = 0.1267
    Lag 21: p-value = 0.1109
    Lag 22: p-value = 0.1188
    Lag 23: p-value = 0.1489
    Lag 24: p-value = 0.1677
  Causalité de Granger significative (p < 0.05) détectée aux lags: [2, 4, 6, 7, 8, 9, 10, 11, 12, 14]

--- Test de Causalité de Granger : Taux d'Intérêt -> Gas Stations (max_lag=24) ---
  P-values pour le F-test (Taux d'Intérêt Granger-causant 

/home/raouf/Bureau/TP S2 DAC/DALAS/TEMP_DALAS_NO_NAME/.venv/lib/python3.12/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results

/home/raouf/Bureau/TP S2 DAC/DALAS/TEMP_DALAS_NO_NAME/.venv/lib/python3.12/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results

/home/raouf/Bureau/TP S2 DAC/DALAS/TEMP_DALAS_NO_NAME/.venv/lib/python3.12/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results

/home/raouf/Bureau/TP S2 DAC/DALAS/TEMP_DALAS_NO_NAME/.venv/lib/python3.12/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results

/home/raouf/Bureau/TP S2 DAC/DALAS/TEMP_DALAS_NO_NAME/.venv/lib/python3.12/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning:

verbose is deprecated since functions should

  P-values pour le F-test (Taux d'Intérêt Granger-causant Supermarkets):
    Lag 1: p-value = 0.6458
    Lag 2: p-value = 0.8325
    Lag 3: p-value = 0.7655
    Lag 4: p-value = 0.8914
    Lag 5: p-value = 0.5371
    Lag 6: p-value = 0.4164
    Lag 7: p-value = 0.5025
    Lag 8: p-value = 0.4425
    Lag 9: p-value = 0.5442
    Lag 10: p-value = 0.4647
    Lag 11: p-value = 0.3222
    Lag 12: p-value = 0.0381
    Lag 13: p-value = 0.0077
    Lag 14: p-value = 0.0137
    Lag 15: p-value = 0.0295
    Lag 16: p-value = 0.0306
    Lag 17: p-value = 0.0366
    Lag 18: p-value = 0.0539
    Lag 19: p-value = 0.0792
    Lag 20: p-value = 0.0504
    Lag 21: p-value = 0.0528
    Lag 22: p-value = 0.0742
    Lag 23: p-value = 0.1305
    Lag 24: p-value = 0.1401
  Causalité de Granger significative (p < 0.05) détectée aux lags: [12, 13, 14, 15, 16, 17]

--- Test de Causalité de Granger : Taux d'Intérêt -> Hobby Toy Game (max_lag=24) ---
  P-values pour le F-test (Taux d'Intérêt Granger-causant Hobb

/home/raouf/Bureau/TP S2 DAC/DALAS/TEMP_DALAS_NO_NAME/.venv/lib/python3.12/site-packages/statsmodels/tsa/stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results



### Interprétation des Résultats des Tests de Causalité de Granger

Les tests de causalité de Granger ont été effectués pour évaluer si les variations passées des taux d'intérêt ont un pouvoir prédictif sur les ventes actuelles dans les différentes catégories, en utilisant les décalages optimaux identifiés précédemment (ou un `max_lag` pertinent basé sur l'analyse des décalages). Les séries ont été différenciées d'ordre 1 pour aider à assurer la stationnarité, une condition requise pour ces tests.

Voici une synthèse des résultats basés sur les p-values des tests F (où une p-value < 0.05 indique une causalité de Granger significative) :

*   **Vêtements (Clothing) :** Les taux d'intérêt Granger-causent les ventes de vêtements. Des p-values significatives (< 0.05) ont été observées pour plusieurs décalages, notamment `[2, 4, 6, 7, 8, 9, 10, 11, 12, 14]`, avec une p-value minimale de `0.0128`. Cela indique que les taux d'intérêt passés (surtout à ces décalages) contribuent à prédire les ventes de vêtements.
*   **Stations-Service (Gas Stations) :** Les taux d'intérêt Granger-causent les ventes des stations-service. Des p-values significatives ont été trouvées pour les décalages `[2, 3, 5, 6, 8]`, avec une p-value minimale de `0.0106`.
*   **Électronique (Electronics) :** Forte évidence de causalité de Granger des taux d'intérêt vers les ventes d'électronique pour les décalages testés `[2, 3, 4, 5, 6]` (le `max_lag` utilisé était 6, basé sur l'analyse de corrélation décalée). La p-value minimale est très faible (`0.0000`).
*   **Bijouterie (Jewelry) :** Les taux d'intérêt Granger-causent les ventes de bijoux. Des p-values significatives sont observées pour les décalages `[2, 3, 4, 5, 6, 7, 8, 9, 10]`, avec une p-value minimale de `0.0004`.
*   **Restaurants :** Très forte indication de causalité de Granger des taux d'intérêt vers les ventes des restaurants. Des p-values significatives sont présentes pour une large plage de décalages allant de 2 à 24 mois (`[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]`), avec une p-value minimale de `0.0001`.
*   **Supermarchés (Supermarkets) :** Les taux d'intérêt Granger-causent les ventes des supermarchés, particulièrement pour les décalages `[12, 13, 14, 15, 16, 17]`, avec une p-value minimale de `0.0077`.
*   **Loisirs, Jouets, Jeux (Hobby Toy Game) :** Les taux d'intérêt Granger-causent les ventes dans cette catégorie, avec des p-values significatives pour les décalages `[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]`, et une p-value minimale de `0.0007`.

**Conclusion Générale des Tests de Granger :**

Pour toutes les catégories de ventes analysées, les tests de causalité de Granger indiquent que les variations passées des taux d'intérêt (après différenciation) ont un pouvoir prédictif statistiquement significatif sur les ventes actuelles (également après différenciation). Les décalages spécifiques pour lesquels cette causalité est la plus forte varient d'une catégorie à l'autre, ce qui est cohérent avec les différentes dynamiques de marché et sensibilités des consommateurs.
Ces résultats renforcent l'idée que les taux d'intérêt sont une variable exogène importante à considérer lors de la modélisation et de la prévision des tendances de consommation. Il est important de noter que la causalité de Granger est une forme de causalité statistique (prédictive) et n'implique pas nécessairement une relation de cause à effet directe au sens économique strict, mais elle souligne une interdépendance temporelle significative.

**Note sur les Avertissements (`FutureWarning`) :**

L'exécution des tests a généré des `FutureWarning` concernant le paramètre `verbose` de la fonction `grangercausalitytests` de la bibliothèque `statsmodels`. Ces avertissements signalent une dépréciation future de ce paramètre par les développeurs de la bibliothèque, mais ils n'affectent pas la validité ou l'exécution des résultats obtenus dans ce notebook. Le code fonctionne comme prévu avec la version actuelle de la bibliothèque.

## 9. Ajustement des Ventes en Termes Réels (Ajustées de l'Inflation)
Pour analyser les ventes en termes réels, nous allons utiliser l'indice des prix à la consommation (CPI) pour déflater les séries de ventes nominales. 

In [9]:
# 7.1. Chargement et préparation de l'IPC (CPI)
cpi_file = 'data/cpi-fed.csv'  # ou 'data/cpi-fed.csv' selon disponibilité
df_cpi = pd.read_csv(cpi_file)

# On suppose que la colonne de date s'appelle 'DATE' et la valeur CPI 'CPI'
df_cpi['DATE'] = pd.to_datetime(df_cpi['observation_date'])
df_cpi = df_cpi.set_index('DATE')

# Normalisation de l'indice à 100 pour la première date utile
cpi_index = df_cpi['CPIAUCSL'] / df_cpi['CPIAUCSL'].iloc[0] * 100

# 7.2. Fusion avec df_analysis
df_real = df_analysis.join(cpi_index.rename('CPI_index'), how='left')

# 7.3. Calcul des ventes réelles pour chaque catégorie
sales_cols = [col for col in df_analysis.columns if col.startswith('sales_')]
for col in sales_cols:
    real_col = col + '_real'
    df_real[real_col] = df_real[col] / (df_real['CPI_index'] / 100)

# 7.4. Affichage d'un aperçu
print('Aperçu des ventes nominales et réelles :')
print(df_real[[*sales_cols, *(col + '_real' for col in sales_cols)]].head())

# 7.5. Exemple de visualisation pour une catégorie
import plotly.express as px
fig = px.line(df_real, x=df_real.index, y=['sales_electronics', 'sales_electronics_real'],
              title='Évolution des Ventes Électronique : Nominal vs. Réel')
fig.show()

Aperçu des ventes nominales et réelles :
            sales_clothing  sales_gas_stations  sales_electronics  \
DATE                                                                
2001-01-01         14135.0             21931.0             7376.0   
2001-02-01         14166.0             21615.0             7334.0   
2001-03-01         13801.0             20652.0             7267.0   
2001-04-01         14217.0             21618.0             7212.0   
2001-05-01         13950.0             22628.0             7135.0   

            sales_jewelry  sales_restaurants  sales_supermarkets  \
DATE                                                               
2001-01-01         2062.0            20613.0             31657.0   
2001-02-01         2057.0            20547.0             29913.0   
2001-03-01         2062.0            23239.0             33084.0   
2001-04-01         2016.0            22217.0             31911.0   
2001-05-01         1957.0            23460.0             34154.0   

## 10. Visualisation des Ventes Nominales vs Réelles
Après avoir calculé les séries de ventes ajustées de l'inflation, nous allons comparer pour chaque catégorie la série nominale et la série réelle afin de visualiser l'effet de déflation des données.

In [10]:
# Boucle de visualisation nominal vs réel pour chaque catégorie de ventes
sales_nominal = [col for col in df_real.columns if col.startswith('sales_') and not col.endswith('_real')]  # colonnes nominales
for col in sales_nominal:
    real_col = col + '_real'
    if real_col not in df_real.columns:
        continue  # passe si la colonne réelle n'existe pas
    # Tracé comparatif nominal vs réel
    fig = make_subplots(specs=[[{'secondary_y': False}]], shared_xaxes=True)
    fig.add_trace(go.Scatter(x=df_real.index, y=df_real[col], mode='lines', name=f'Nominal ({col})'))
    fig.add_trace(go.Scatter(x=df_real.index, y=df_real[real_col], mode='lines', name=f'Réel ({real_col})'))
    fig.update_layout(
        title=f'Évolution des Ventes : {col.replace('sales_','').replace('_',' ').title()} - Nominal vs Réel',
        xaxis_title='Date',
        yaxis_title='Ventes (Millions USD)',
    )
    fig.show()

## 11. Matrice de Corrélation sur les Ventes Réelles
Nous allons calculer et visualiser la corrélation entre le taux d'intérêt et les séries de ventes ajustées (réelles) pour chaque catégorie.

In [11]:
# Sélection des colonnes de ventes réelles et du taux d'intérêt
real_sales_cols = [col for col in df_real.columns if col.endswith('_real')]
# Calcul de la matrice de corrélation
corr_real = df_real[['interest_rate'] + real_sales_cols].corr()
# Affichage de la heatmap de corrélation
import plotly.express as px
fig_corr_real = px.imshow(corr_real,
                       text_auto=True,
                       aspect='auto',
                       color_continuous_scale='RdBu_r',
                       title="Matrice de Corrélation: Taux d'Intérêt vs Ventes Réelles")
fig_corr_real.show()
# Affichage des valeurs numériques
print("Matrice de corrélation (valeurs numériques) :")
print(corr_real)

Matrice de corrélation (valeurs numériques) :
                           interest_rate  sales_clothing_real  \
interest_rate                   1.000000             0.054108   
sales_clothing_real             0.054108             1.000000   
sales_gas_stations_real        -0.042941             0.435204   
sales_electronics_real          0.085963             0.185080   
sales_jewelry_real              0.296998             0.624160   
sales_restaurants_real          0.157194             0.265215   
sales_supermarkets_real        -0.031805            -0.093831   
sales_hobby_toy_game_real       0.000548             0.069876   

                           sales_gas_stations_real  sales_electronics_real  \
interest_rate                            -0.042941                0.085963   
sales_clothing_real                       0.435204                0.185080   
sales_gas_stations_real                   1.000000               -0.162974   
sales_electronics_real                   -0.162974      

## 9.1. Interprétation de la Matrice de Corrélation sur les Ventes Réelles
*Note : Les décalages optimaux identifiés pour les données nominales (ex: 6 mois pour l’électronique) ont évolué pour les séries réelles (ex: 24 mois pour l’électronique), en raison de l’ajustement à l’inflation.*
Voici la matrice de corrélation entre le taux d'intérêt et les séries de ventes ajustées de l'inflation :

| Catégorie                  | Corrélation avec le Taux (Réel) | Corrélation Nominale | Changement de signe/magnitude |
|-----------------------------|:------------------------------:|:--------------------:|:-----------------------------:|
| Vêtements                   | 0.0541                         | 0.0897               | Baisse, reste positive        |
| Stations-Service            | -0.0429                        | 0.0339               | Inversion de signe            |
| Électronique                | 0.0860                         | 0.1170               | Légère baisse                 |
| Bijouterie                  | 0.2970                         | 0.2170               | Augmentation                  |
| Restaurants                 | 0.1572                         | 0.1860               | Baisse                         |
| Supermarchés                | -0.0318                        | 0.0680               | Inversion de signe            |
| Jouets/Loisirs              | 0.0005                         | 0.0226               | Baisse, quasiment nul         |

**Points-clés :**
1. La plupart des corrélations nominales s’affaiblissent après ajustement pour l’inflation, voire s’inversent (stations-service, supermarchés).
2. Les ventes de bijoux voient leur corrélation avec les taux fortement augmenter (de 0.217 à 0.297), suggérant un effet inflation corrigé plus net pour le luxe.
3. Les relations perçues auparavant pouvaient être en partie dues à l’évolution générale des prix ; les séries réelles isolent la demande réelle des effets de niveau de prix.
4. Certains secteurs (électronique, restaurants) conservent une corrélation modérée mais réduite, indiquant que l’inflation masquait partiellement la vraie sensibilité.

**Prochaine étape :** relancer l’analyse des décalages temporels (Lag Analysis) en utilisant les séries réelles (`sales_*_real`) pour identifier si les délais et amplitudes des effets sur la demande changent lorsqu’on travaille en termes réels.

## 12. Lag Analysis sur les Ventes Réelles
Pour vérifier si l’ajustement en termes réels modifie les délais et la force des corrélations, nous relançons l’analyse des décalages temporels en utilisant les colonnes `sales_*_real`.

In [12]:
from scipy.stats import pearsonr
import numpy as np
import plotly.express as px

# Préparation des séries réelles et du taux d'intérêt
df_lag_real = df_real.copy()  # contient interest_rate et sales_*_real
sales_real_cols = [col for col in df_lag_real.columns if col.endswith('_real')]
# Définir la plage de décalages à tester (en mois)
max_lag = 24
lags = range(0, max_lag + 1)
lag_results_real = {}
print(f'Analyse des décalages (0-{max_lag} mois) sur ventes réelles :')
for col in sales_real_cols:
    cors = []
    for lag in lags:
        shifted = df_lag_real['interest_rate'].shift(lag)
        tmp = pd.DataFrame({ 'sales': df_lag_real[col], 'rate': shifted }).dropna()
        if len(tmp) < 2:
            cors.append(np.nan)
        else:
            c, _ = pearsonr(tmp['sales'], tmp['rate'])
            cors.append(c)
    lag_results_real[col] = pd.Series(cors, index=lags)
    best = lag_results_real[col].abs().idxmax()
    val = lag_results_real[col].loc[best]
    name = col.replace('_real','').replace('sales_','').replace('_',' ').title()
    print(f'{name}: meilleur lag = {best} mois, corr = {val:.4f}')
    # Visualisation
    fig = px.bar(lag_results_real[col], title=f'Corrélation Croisée Réelle: {name}', labels={'index':'Lag (mois)','value':'Corrélation'})
    fig.add_vline(x=best, line_dash='dash', line_color='red')
    fig.show()
# Résumé tabulaire
summary = {
    'Catégorie': [],
    'Best Lag (mois)': [],
    'Corr Best': []
}
for col, ser in lag_results_real.items():
    lag0 = ser.abs().idxmax()
    summary['Catégorie'].append(col.replace('_real','').replace('sales_','').title())
    summary['Best Lag (mois)'].append(lag0)
    summary['Corr Best'].append(f"{ser.loc[lag0]:.4f}")
df_summary_real = pd.DataFrame(summary)
print('\nRésumé Lag Analysis Ventes Réelles:')
print(df_summary_real.to_string(index=False))

Analyse des décalages (0-24 mois) sur ventes réelles :
Clothing: meilleur lag = 24 mois, corr = -0.2618


Gas Stations: meilleur lag = 23 mois, corr = -0.3018


Electronics: meilleur lag = 24 mois, corr = 0.3439


Jewelry: meilleur lag = 0 mois, corr = 0.2970


Restaurants: meilleur lag = 24 mois, corr = -0.3929


Supermarkets: meilleur lag = 24 mois, corr = -0.3257


Hobby Toy Game: meilleur lag = 22 mois, corr = 0.0868



Résumé Lag Analysis Ventes Réelles:
     Catégorie  Best Lag (mois) Corr Best
      Clothing               24   -0.2618
  Gas_Stations               23   -0.3018
   Electronics               24    0.3439
       Jewelry                0    0.2970
   Restaurants               24   -0.3929
  Supermarkets               24   -0.3257
Hobby_Toy_Game               22    0.0868


## 13. Modélisation par Régression Linéaire avec Décalages Optimaux
Nous allons construire pour chaque catégorie de ventes réelles un modèle de régression linéaire simple :

 sales_real(t) = alpha + beta * interest_rate(t - lag_opt) + epsilon,

où lag_opt est le décalage optimal identifié précédemment. Nous examinerons les coefficients, R² et la significativité de beta.

### 11.1. Interprétation des Modèles de Régression Linéaire

Pour chaque catégorie de ventes réelles, nous avons estimé un modèle simple  
  sales_real(t) = α + β · interest_rate(t – lag_opt) + ε  
en utilisant le décalage optimal identifié précédemment. Voici les points clés :

| Catégorie            | Lag utilisé | R-squared | β (coef)     | P-value (β) | Interprétation rapide                                 |
|----------------------|:-----------:|:---------:|:------------:|:-----------:|:-------------------------------------------------------|
| **Clothing**         | 24 mois     | 0.069     | −25.65       | <0.001      | Relation négative modérée : une hausse de 1 pt de taux → −25.6 M$ de ventes réelles 2 ans plus tard. Significatif. |
| **Gas Stations**     | 23 mois     | 0.091     | −94.46       | <0.001      | Relation négative modérée : 1 pt ↦ −94.5 M$ de ventes de carburant. Significatif.​ |
| **Electronics**      | 24 mois     | 0.118     | +31.49       | <0.001      | Relation positive : 1 pt ↦ +31.5 M$ d’électronique après 2 ans. Significatif et plus fort qu’en nominal. |
| **Jewelry**          | 0 mois      | 0.088     | +3.50        | <0.001      | Effet instantané positif : +3.5 M$ de bijoux par pt de taux. Significatif.  |
| **Restaurants**      | 24 mois     | 0.154     | −215.86      | <0.001      | Relation négative la plus forte : −215.9 M$ ; sensibilité élevée des restaurants après 2 ans. |
| **Supermarkets**     | 24 mois     | 0.106     | −81.29       | <0.001      | Relation négative modérée : −81.3 M$ de ventes alimentaires pour 1 pt de taux. |
| **Hobby Toy Game**   | 22 mois     | 0.008     | +3.20        | 0.159       | Effet non significatif (P>0.05). Peu de pouvoir explicatif (R²≈0). |

**Principales conclusions :**  
- Tous les coefficients β (sauf HobbyToyGame) sont significatifs à 1 % et confirment une influence des taux d’intérêt sur la consommation réelle.  
- Les secteurs « Restaurants » et « Gas Stations » présentent les sensibilités absolues les plus élevées (|β|>90 M$).  
- L’électronique et la bijouterie dérogent à la tendance négative : elles réagissent positivement aux hausses de taux (décalages différents).  
- Les R² restent modestes (0.07–0.15) : d’autres facteurs macroéconomiques expliquent une grande partie de la variation des ventes réelles.  
- Le cas « Hobby Toy Game » n’est pas capté par ce simple modèle linéaire retardé.  


## 14. Conclusion Générale de l'Analyse

Ce notebook a entrepris une analyse approfondie de l'impact des taux d'intérêt sur diverses catégories de ventes au détail. L'objectif était de comprendre la nature, la direction, et le timing de cette relation.

**1. Préparation des Données :**
Les données de taux d'intérêt (Federal Funds Effective Rate) et de ventes au détail pour sept catégories (Vêtements, Stations-Service, Électronique, Bijouterie, Restaurants, Supermarchés, Loisirs/Jouets/Jeux) ont été chargées, nettoyées, et fusionnées pour la période de janvier 2001 à novembre 2024. Les valeurs manquantes ont été traitées par la méthode `ffill`.

**2. Analyse Exploratoire Initiale (Données Nominales) :**
L'analyse visuelle et la matrice de corrélation sur les données nominales ont révélé des corrélations directes (lag 0) généralement faibles entre les taux d'intérêt et les ventes. Cependant, de fortes corrélations positives ont été observées *entre* les différentes catégories de ventes, suggérant l'influence de facteurs macroéconomiques communs.

**3. Analyse des Décalages (Données Nominales) :**
L'analyse des décalages a montré que pour la plupart des catégories de ventes nominales (Vêtements, Stations-Service, Restaurants, Supermarchés, Bijouterie), la corrélation la plus forte avec les taux d'intérêt était négative et se manifestait avec un décalage de 24 mois. Les ventes d'électronique faisaient exception avec une corrélation positive faible à un décalage de 6 mois.

**4. Tests de Causalité de Granger (Données Nominales Différenciées) :**
Après différenciation pour assurer la stationnarité, les tests de causalité de Granger ont indiqué que les variations passées des taux d'intérêt ont un pouvoir prédictif statistiquement significatif sur les variations actuelles des ventes pour toutes les catégories analysées. Les décalages significatifs variaient, confirmant l'influence temporelle des taux.

**5. Passage aux Données Réelles :**
Pour neutraliser l'effet de l'inflation, les séries de ventes nominales ont été ajustées à l'aide de l'Indice des Prix à la Consommation (IPC), créant des séries de ventes réelles.

**6. Analyse sur les Données Réelles :**
*   **Corrélations Directes :** La matrice de corrélation sur les ventes réelles a montré des changements notables : certaines corrélations se sont affaiblies, d'autres ont changé de signe (ex: Stations-Service, Supermarchés devenant négatives), et celle de la Bijouterie s'est renforcée positivement (0.297 à lag 0).
*   **Analyse des Décalages (Données Réelles) :** Les décalages optimaux ont également évolué. Pour les ventes réelles :
    *   Vêtements : -0.2618 (lag 24)
    *   Stations-Service : -0.3018 (lag 23)
    *   Électronique : +0.3439 (lag 24) - changement notable par rapport au nominal.
    *   Bijouterie : +0.2970 (lag 0) - effet instantané.
    *   Restaurants : -0.3929 (lag 24)
    *   Supermarchés : -0.3257 (lag 24)
    *   Loisirs/Jouets/Jeux : +0.0868 (lag 22) - corrélation faible.

**7. Modélisation par Régression Linéaire Simple (Données Réelles avec Lags Optimaux) :**
Des modèles de régression linéaire simple (Ventes Réelles(t) ~ Taux d'Intérêt(t-lag_opt)) ont été estimés :
*   Les coefficients (β) pour le taux d'intérêt étaient statistiquement significatifs (p < 0.001) pour la plupart des catégories, confirmant une influence des taux sur la consommation réelle.
*   Les secteurs des **Restaurants** (β ≈ -216M$) et des **Stations-Service** (β ≈ -94M$) ont montré la plus forte sensibilité négative en termes absolus.
*   L'**Électronique** (β ≈ +31M$) et la **Bijouterie** (β ≈ +3.5M$) ont montré une relation positive significative.
*   Les **Vêtements** (β ≈ -26M$) et les **Supermarchés** (β ≈ -81M$) ont également montré des relations négatives significatives.
*   Le secteur **Loisirs/Jouets/Jeux** n'a pas montré de relation significative dans ce modèle simple (p > 0.05).
*   Les valeurs de R-squared étaient modestes (généralement entre 0.07 et 0.15), indiquant que si les taux d'intérêt ont un impact, une grande partie de la variation des ventes réelles est expliquée par d'autres facteurs non inclus dans ces modèles simples.


**Conclusion Finale :**
L'analyse confirme que les taux d'intérêt exercent une influence statistiquement significative sur la consommation des ménages dans la plupart des secteurs étudiés, bien que cette influence soit complexe, souvent retardée, et variable selon les catégories de biens et services. L'impact est plus clairement discerné lorsque l'on considère les ventes en termes réels et que l'on tient compte des décalages temporels appropriés.

Les secteurs de biens durables ou de dépenses discrétionnaires importantes (comme les restaurants) tendent à montrer des sensibilités plus marquées. Les relations positives observées pour l'électronique et la bijouterie (en termes réels et avec décalages) sont des résultats notables qui mériteraient une investigation plus approfondie pour en comprendre les mécanismes sous-jacents (par exemple, cycles de produits, comportement d'investissement en période d'incertitude, ou effets de richesse spécifiques).

La modestie des R-squared dans les modèles de régression simple souligne que les taux d'intérêt ne sont qu'un des nombreux facteurs influençant la consommation. Des modèles plus sophistiqués, intégrant d'autres variables macroéconomiques (PIB, chômage, confiance des consommateurs, etc.) et les interdépendances entre secteurs (comme dans un modèle VAR/VECM), sont nécessaires pour une compréhension plus complète et un meilleur pouvoir explicatif.